## Fake Data Detection


### About The Data
This dataset contains 18K job descriptions out of which about 800 are fake. The data consists of both textual information and meta-information about the jobs. The dataset can be used to create classification models which can learn the job descriptions which are fraudulent. A small proportion of these descriptions are fake or scam which can be identified by the column "fraudulent". 

The data is provide by the University of the Aegean | Laboratory of Information & Communication Systems Security

http://emscad.samos.aegean.gr/

In [2]:
# Install dependencies as needed:
import pandas as pd 


### Dictonary:

-  job_id: Unique ID (int64)
-  title: Title of job description (str)
-  location: Geographical location of the job ad (Example: US, NY, New York)
-  department: Corporate department (e.g. Marketing, Success, Sales, ANDROIDPIT, ...)
-  salary_range: Indicative salary range (e.g. 50,000-60,000 ($))
-  company_profile: A brief company description.
-  description: The details description of the job ad.
-  requirements: Enlisted requirements for the job opening.
-  benefits: Enlisted offered benefits by the employer.
-  telecommuting: True for telecommuting positions.
-  has_company_logo: True if company logo is present.
-  has_questions: True if screening questions are present.
-  employment_type: Type of emplyment (e.g. Full-type, Part-time, Contract, etc.)
-  required_experience: Required Experience (e.g. Executive, Entry level, Intern, etc.)
-  required_education: Required Education (e.g. Doctorate, Master’s Degree, Bachelor, etc)
-  industry: Industry (e.g. Automotive, IT, Health care, Real estate, etc.)
-  function: Position as function in the company (e.g. Consulting, Engineering, Research, Sales etc.)
-  fraudulent: Classifcation target (0, 1)

In [3]:
# read file
data_path = 'D:/01_projects/09_LeWagon/FakeJobPosting/data/fake_job_postings.csv'
df = pd.read_csv(data_path)
# print("First 5 records:", df.head())

In [4]:
df.dtypes

job_id                  int64
title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: object

In [6]:
print(f'''
Shape: {df.shape}
Size: {df.size}
Unique Ids: {df.job_id.nunique()}
Locations: {df.location.nunique()}
Departments: {df.department.nunique()}; {df.department.unique()}
Salary Range: {df.salary_range.describe()}
Column names: {df.columns}

'''
)



Shape: (17880, 18)
Size: 321840
Unique Ids: 17880
Locations: 3105
Departments: 1337; ['Marketing' 'Success' nan ... 'Admin - Clerical' 'Administrative Dept'
 'Hospitality']
Salary Range: count     2868
unique     874
top        0-0
freq       142
Name: salary_range, dtype: object
Column names: Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')




# Thoughts

- For the basic ML Pipeline, we vectorize each of the descriptions seperately, add these vectors along with metainformation to train the model.
- For deep learning, we will create different paths
    1. Creating one document per entry by merging all information in a systematic way.
    2. Creating a meta-data sentence, and leave all description parts seperated. We will then train one moddel per part. At theend we will train a model taking the probabilities for each part to get to a desion.
    3. Same as 2. just that we will train one model taking all parts as inputs
